<h1> Coursea Capstone </h1>

<h1>IBM Applied Data Science Capstone</h1>

<h6>Opening a new shopping mall in New York,NY</h6>

<h6>Build a dataframe of neighbourhood in New York,NY by web scraping the data from Wikipedia page</h6>

<h6>Get the geographical co-ordinated of the neighbourhoods</h6>

<h6>Obtain the venue data for the neighbourhoods from Foursquare API</h6>

<h6>Explore and cluster the neighbourhoods</h6>

<h6>Select the best cluster to open a new shopping mall</h6>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = ''
CLIENT_SECRET= ''
VERSION = '20180604'
LIMIT = 30

In [3]:
address = 'New York,NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


#### Send GET request and examine results

In [3]:
import requests

In [4]:
import json # library to handle JSON files


In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [46]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)



In [47]:
neighborhoods_data = newyork_data['features']

In [48]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [49]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [50]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [51]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [52]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [53]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [54]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [55]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [56]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


Let's get the geographical coordinates of Manhattan.

In [57]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


As we did with all of New York City, let's visualizat Brooklyn the neighborhoods in it.

In [58]:
# create map of Brookyln using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

#### Let's explore the first neighborhood in our dataframe.

In [59]:
brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

Get the neighborhood's latitude and longitude values.

In [60]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [61]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=O5BIGTQRSHFEK4HOODQHOJXXICVXDURRU451FQUTF0X1YWL4&client_secret=Q14VNBMMYV2GR0NX0A42QXS2UPYZYYI05CGB4MFMTT1DQQIF&v=20180604&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [63]:
results = requests.get(url).json()


In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [65]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Cocoa Grinder,Juice Bar,40.623967,-74.030863
3,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
4,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371


In [66]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

83 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Brooklyn

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )


Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [69]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2801, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [70]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,46,46,46,46,46,46
Bay Ridge,83,83,83,83,83,83
Bedford Stuyvesant,26,26,26,26,26,26
Bensonhurst,30,30,30,30,30,30
Bergen Beach,7,7,7,7,7,7
Boerum Hill,87,87,87,87,87,87
Borough Park,22,22,22,22,22,22
Brighton Beach,44,44,44,44,44,44
Broadway Junction,16,16,16,16,16,16


#### Let's find out how many unique categories can be curated from all the returned venues

In [71]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 285 uniques categories.


## 3. Analyze Each Neighborhood

In [72]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [73]:
brooklyn_onehot.shape

(2801, 285)

In [74]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bath Beach,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.021739,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.021739
1,Bay Ridge,0.000000,0.00,0.00,0.000000,0.036145,0.000000,0.00,0.00,0.012048,...,0.000000,0.012048,0.000000,0.012048,0.00,0.000000,0.000000,0.000000,0.00,0.000000
2,Bedford Stuyvesant,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.038462,0.00,0.000000,0.038462,0.038462,0.00,0.000000
3,Bensonhurst,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
4,Bergen Beach,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
5,Boerum Hill,0.022989,0.00,0.00,0.000000,0.011494,0.011494,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.011494,0.00,0.000000
6,Borough Park,0.000000,0.00,0.00,0.000000,0.045455,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
7,Brighton Beach,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
8,Broadway Junction,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
9,Brooklyn Heights,0.050000,0.00,0.00,0.000000,0.020000,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.010000,0.020000,0.00,0.010000


In [75]:
brooklyn_grouped.shape

(70, 285)

In [76]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0              Pharmacy  0.09
1           Pizza Place  0.04
2       Bubble Tea Shop  0.04
3  Fast Food Restaurant  0.04
4    Chinese Restaurant  0.04


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.07
1                  Spa  0.07
2          Pizza Place  0.04
3     Greek Restaurant  0.04
4  American Restaurant  0.04


----Bedford Stuyvesant----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.08
2                     Bar  0.08
3             Pizza Place  0.08
4  Thrift / Vintage Store  0.04


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.17
1          Donut Shop  0.07
2  Italian Restaurant  0.07
3    Sushi Restaurant  0.07
4      Ice Cream Shop  0.07


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.29
1  Athletics & Sports  0.14
2      Baseball Field  0.14
3                Park  0.14
4          Playground  0.14


---

In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Fast Food Restaurant,Donut Shop,Italian Restaurant,Kids Store,Chinese Restaurant,Bubble Tea Shop,Pizza Place,Peruvian Restaurant,Gas Station
1,Bay Ridge,Italian Restaurant,Spa,Greek Restaurant,Bar,American Restaurant,Thai Restaurant,Bagel Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant
2,Bedford Stuyvesant,Café,Pizza Place,Bar,Coffee Shop,Deli / Bodega,BBQ Joint,Fried Chicken Joint,Sandwich Place,Boutique,Fruit & Vegetable Store
3,Bensonhurst,Chinese Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Playground,Cosmetics Shop,Pizza Place,Dumpling Restaurant,Noodle House
4,Bergen Beach,Harbor / Marina,Donut Shop,Baseball Field,Playground,Athletics & Sports,Park,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [79]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 2, 0, 2, 2, 4, 2, 4, 2], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Italian Restaurant,Spa,Greek Restaurant,Bar,American Restaurant,Thai Restaurant,Bagel Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0,Chinese Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Playground,Cosmetics Shop,Pizza Place,Dumpling Restaurant,Noodle House
2,Brooklyn,Sunset Park,40.645103,-74.010316,4,Latin American Restaurant,Bakery,Bank,Pizza Place,Mexican Restaurant,Gym,Pharmacy,Mobile Phone Shop,Sandwich Place,Record Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Yoga Studio,French Restaurant,Café,Boutique,Mexican Restaurant,Record Shop
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Italian Restaurant,Lounge,Chinese Restaurant,Gym,Bakery,Bus Station,Deli / Bodega,Breakfast Spot,Pharmacy


In [81]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [82]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Greek Restaurant,Bar,American Restaurant,Thai Restaurant,Bagel Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant
1,Bensonhurst,Chinese Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Playground,Cosmetics Shop,Pizza Place,Dumpling Restaurant,Noodle House
4,Gravesend,Pizza Place,Italian Restaurant,Lounge,Chinese Restaurant,Gym,Bakery,Bus Station,Deli / Bodega,Breakfast Spot,Pharmacy
20,Carroll Gardens,Italian Restaurant,Coffee Shop,Cocktail Bar,Pizza Place,Wine Shop,Bakery,Bar,Gym / Fitness Center,Spa,Gift Shop
38,Clinton Hill,Italian Restaurant,Pizza Place,Mexican Restaurant,Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Yoga Studio,Wine Shop,Chinese Restaurant
42,Prospect Lefferts Gardens,Café,Bakery,Caribbean Restaurant,Pizza Place,Liquor Store,Sushi Restaurant,Supermarket,Indian Restaurant,Ice Cream Shop,Wine Shop
46,Midwood,Pizza Place,Ice Cream Shop,Pharmacy,Convenience Store,Video Game Store,Italian Restaurant,Bank,Bakery,Candy Store,Filipino Restaurant
52,Ocean Parkway,Grocery Store,Bakery,Pizza Place,Gym / Fitness Center,Playground,Steakhouse,Sake Bar,General Entertainment,Gift Shop,Bus Station
60,Mill Basin,Pizza Place,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Bank,Chinese Restaurant,Bagel Shop,Burger Joint,Supermarket,Breakfast Spot
68,Madison,Bagel Shop,Deli / Bodega,Italian Restaurant,Bus Station,Restaurant,Spa,Dessert Shop,Pilates Studio,Pizza Place,Dim Sum Restaurant


In [83]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Gym,Home Service,Chinese Restaurant,Caribbean Restaurant,Asian Restaurant,Grocery Store,Flea Market,Flower Shop,Fish Market,Fish & Chips Shop
59,Paerdegat Basin,Gym,Home Service,Asian Restaurant,Child Care Service,Chinese Restaurant,Food,Moving Target,Flower Shop,Flea Market,Fish Market


In [84]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Greenpoint,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Yoga Studio,French Restaurant,Café,Boutique,Mexican Restaurant,Record Shop
5,Brighton Beach,Eastern European Restaurant,Beach,Restaurant,Russian Restaurant,Bank,Sushi Restaurant,Mobile Phone Shop,Fast Food Restaurant,Gourmet Shop,Korean Restaurant
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Pizza Place,Café,Russian Restaurant,Restaurant,Hotel,Department Store
9,Crown Heights,Pizza Place,Café,Moving Target,Museum,Metro Station,Bagel Shop,Candy Store,Bookstore,Supermarket,Salon / Barbershop
11,Kensington,Grocery Store,Thai Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Sandwich Place,Pharmacy,Mobile Phone Shop,Gas Station,Lingerie Store
12,Windsor Terrace,Café,Diner,Park,Grocery Store,Bagel Shop,Moving Target,Bookstore,Chinese Restaurant,Sushi Restaurant,Beer Store
13,Prospect Heights,Bar,Mexican Restaurant,Ice Cream Shop,Thai Restaurant,Wine Shop,Bakery,Cocktail Bar,Gourmet Shop,American Restaurant,Beer Bar
14,Brownsville,Restaurant,Moving Target,Park,Performing Arts Venue,Caribbean Restaurant,Chinese Restaurant,Fried Chicken Joint,Farmers Market,Trail,Discount Store
15,Williamsburg,Bar,Bagel Shop,Coffee Shop,Food Truck,Taco Place,Korean Restaurant,Latin American Restaurant,Steakhouse,Liquor Store,Lounge
16,Bushwick,Bar,Coffee Shop,Mexican Restaurant,Deli / Bodega,Pizza Place,Discount Store,Bakery,Thrift / Vintage Store,New American Restaurant,Italian Restaurant


In [85]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Food,Lake,Food Court,Food & Drink Shop,Food Stand,Flower Shop,Flea Market,Fish Market,Event Space


In [86]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Latin American Restaurant,Bakery,Bank,Pizza Place,Mexican Restaurant,Gym,Pharmacy,Mobile Phone Shop,Sandwich Place,Record Shop
7,Manhattan Terrace,Pizza Place,Pharmacy,Donut Shop,Ice Cream Shop,Bagel Shop,Burger Joint,Steakhouse,Chinese Restaurant,Bus Station,Grocery Store
8,Flatbush,Mexican Restaurant,Deli / Bodega,Caribbean Restaurant,Coffee Shop,Donut Shop,Chinese Restaurant,Lounge,Bank,Juice Bar,Bagel Shop
10,East Flatbush,Department Store,Hardware Store,Chinese Restaurant,Food & Drink Shop,Caribbean Restaurant,Bakery,Fast Food Restaurant,Supermarket,Check Cashing Service,Park
25,Cypress Hills,Fried Chicken Joint,Ice Cream Shop,Latin American Restaurant,Donut Shop,Fast Food Restaurant,Spanish Restaurant,Pizza Place,Bank,Discount Store,Food
26,East New York,Deli / Bodega,Bus Station,Fast Food Restaurant,Gym,Music Venue,Caribbean Restaurant,Liquor Store,Latin American Restaurant,Event Service,Spanish Restaurant
27,Starrett City,Pharmacy,Donut Shop,Caribbean Restaurant,Pizza Place,American Restaurant,Supermarket,Shopping Mall,Women's Store,Farm,Farmers Market
29,Flatlands,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Deli / Bodega,Fried Chicken Joint,Park,Electronics Store,Lounge,Paper / Office Supplies Store,Bar
32,Coney Island,Baseball Stadium,Deli / Bodega,Caribbean Restaurant,Gourmet Shop,Monument / Landmark,Brewery,Skating Rink,Music Venue,Beach,Theme Park Ride / Attraction
33,Bath Beach,Pharmacy,Fast Food Restaurant,Donut Shop,Italian Restaurant,Kids Store,Chinese Restaurant,Bubble Tea Shop,Pizza Place,Peruvian Restaurant,Gas Station
